In [1]:
# !pip install category_encoders

In [2]:
# Importing Packages
import numpy as np
import pandas as pd 
import keras

from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import category_encoders as ce
from imblearn.over_sampling import SMOTE
from tensorflow.keras import initializers
import seaborn as sns
from sklearn.metrics import classification_report,confusion_matrix

In [4]:
# Importing Data
df_hr = pd.read_csv("HR_comma_sep.csv")

In [5]:
# Data Overview
df_hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,support,low
14995,0.37,0.48,2,160,3,0,1,0,support,low
14996,0.37,0.53,2,143,3,0,1,0,support,low
14997,0.11,0.96,6,280,4,0,1,0,support,low


In [6]:
# One hot encoding
def one_hot_encoding(df,col):
    one_hot_encoder=ce.OneHotEncoder(cols=col,return_df=True,use_cat_names=True)
    df_final = one_hot_encoder.fit_transform(df)
    return df_final

In [7]:
def ordinal_encoding(df,col,mapping):
    ordinal_encoder=ce.OrdinalEncoder(cols=col,return_df=True,mapping=[{'col':col,'mapping':mapping}])
    df_final = ordinal_encoder.fit_transform(df)
    return df_final

In [8]:
df_hr_one_hot = one_hot_encoding(df_hr,"sales")

In [9]:
df_hr_one_hot = one_hot_encoding(df_hr_one_hot,"salary")

In [10]:
df_final = df_hr_one_hot.copy()
X = df_final.drop(["left"],axis = 1)
Y = df_final[["left"]]

In [11]:
Y.value_counts()

left
0       11428
1        3571
dtype: int64

In [12]:
smote = SMOTE()
X, Y = smote.fit_resample(X, Y)

In [13]:
# Plotting accuracy of train,test
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()
    

In [14]:
def train_nn(test_ratio,optimizer,epochs,batch_size):
    X_train, X_test,y_train,y_test = train_test_split(X,Y,test_size = test_ratio,random_state = 27)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            if(logs.get("val_accuracy")>98.2):
                  print("Reached the accuracy required (ie) 90%", logs)
                  self.model.stop_training=True
    callback=myCallback()
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.05))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.05))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer = optimizer,
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])
    history = model.fit(X_train,y_train,epochs=epochs,validation_data=(X_test,y_test),batch_size = batch_size,callbacks=[callback])
    print(f"For {optimizer} with Batch Size {batch_size} \n")
    plot_metric(history,"accuracy")
    
    Y_pred = model.predict(X_test)
    Y_pred = np.round(Y_pred)
    sns.heatmap(confusion_matrix(Y_pred,y_test),annot = True, fmt='g')

In [15]:
Y_pred = train_nn(0.2,"adam",1000,256)

Epoch 1/1000


C:\Users\Karan\miniconda3\envs\cpu\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


72/72 [==============================] - 1s 9ms/step - loss: 0.4257 - accuracy: 0.8062 - val_loss: 0.1981 - val_accuracy: 0.9302
Epoch 2/1000
 8/72 [==>...........................] - ETA: 0s - loss: 0.2290 - accuracy: 0.9150

C:\Users\Karan\miniconda3\envs\cpu\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


72/72 [==============================] - 0s 7ms/step - loss: 0.2019 - accuracy: 0.9317 - val_loss: 0.1745 - val_accuracy: 0.9401
Epoch 3/1000
72/72 [==============================] - 0s 6ms/step - loss: 0.1662 - accuracy: 0.9472 - val_loss: 0.1322 - val_accuracy: 0.9582
Epoch 4/1000
72/72 [==============================] - 0s 7ms/step - loss: 0.1466 - accuracy: 0.9530 - val_loss: 0.1278 - val_accuracy: 0.9593
Epoch 5/1000
72/72 [==============================] - 0s 6ms/step - loss: 0.1396 - accuracy: 0.9549 - val_loss: 0.1321 - val_accuracy: 0.9578
Epoch 6/1000
72/72 [==============================] - 0s 6ms/step - loss: 0.1270 - accuracy: 0.9578 - val_loss: 0.1135 - val_accuracy: 0.9613
Epoch 7/1000
72/72 [==============================] - 0s 6ms/step - loss: 0.1205 - accuracy: 0.9597 - val_loss: 0.1086 - val_accuracy: 0.9637
Epoch 8/1000
72/72 [==============================] - 0s 6ms/step - loss: 0.1181 - accuracy: 0.9609 - val_loss: 0.1053 - val_accuracy: 0.9659
Epoch 9/1000
72/72 